In [1]:
import pandas as pd

# Refining

In [2]:
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
EFF_H2_SMR = 159.6 #MJ/kgH2

In [3]:
lcoh_res_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx')
res_be_df = pd.DataFrame(columns=['RES', 'Breakeven price ($/MMBtu)'])
res_be_df['RES'] = lcoh_res_df['RES']
res_be_df['Breakeven price ($/MMBtu)'] = lcoh_res_df.apply(lambda x:x['LCOH']/(EFF_H2_SMR*CONV_MJ_TO_MMBTU), axis=1)
res_be_df

,RES,Breakeven price ($/MMBtu)
0,Grid PEM,34.837997
1,Wind subsidized,37.250875
2,Wind unsubsidized,43.861501
3,Solar PEC,49.579692
4,Solar PV-E,50.901817
5,SMR 89% CCUS,12.758507


In [4]:
res_be_df.to_csv('./results/res_be_refining.csv', header=True, index=False)


## Steel

In [5]:
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
COAL_CONS_RATE = 0.663 #ton-coal/ton-steel for conventional BF/BOF plant
iron_ore_cost = 100 #$/t_ironore
bfbof_iron_cons = 1.226 #t_ironore/t_steel
om_bfbof = 353.25 #$/t_steel
DRICO2Intensity = 40 # kgCO2/ton-DRI`````````````````
ShaftFCAPEX = 250 # $/tDRI/year
EAFCAPEX = 160 # $/tsteel/year
EAFOM  = 24.89 # $/tsteel (EAF and casting)
Steel = 800 # $/tsteel
RatioSteelDRI = 0.9311 # tsteel/tDRI
RatioIronOreDRI = 1.391 # tironore/tDRI


In [6]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [7]:
steel_df = pd.read_excel('h2_demand_bfbof_steel_us_2022.xlsx', sheet_name='processed')
steel_ids = list(steel_df['Plant'])

In [8]:
res_be_df= pd.DataFrame(columns=['Plant', 'Grid PEM', 'Wind unsubsidized', 'Wind subsidized', 'Solar PEC','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['Plant'] = steel_df['Plant']
res_be_df.set_index('Plant', inplace=True)


In [9]:
from opt_deployment_steel import get_steel_plant_demand
def compute_res_be(lcoh, lcoe, plant):
  steel_cap_ton_per_annum, h2_dem_kg_per_day, elec_dem_MWe = get_steel_plant_demand(plant)
  costs_dri_eaf = steel_cap_ton_per_annum*(EAFCAPEX + ShaftFCAPEX/RatioSteelDRI + EAFOM +\
            iron_ore_cost*RatioIronOreDRI/RatioSteelDRI)
  costs_h2 =  lcoh*h2_dem_kg_per_day*365
  costs_electricity = lcoe*elec_dem_MWe*24*365
  costs_bfbof = iron_ore_cost*bfbof_iron_cons*steel_cap_ton_per_annum + om_bfbof*steel_cap_ton_per_annum
  coal_ann_consumption = steel_cap_ton_per_annum*COAL_CONS_RATE
  res_be = (costs_h2+costs_electricity+costs_dri_eaf - costs_bfbof)/coal_ann_consumption
  return res_be

In [10]:
for plant in steel_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    lcoe = lco_df.loc[res, 'LCOE']
    res_be = compute_res_be(lcoh, lcoe, plant)
    res_be_df.loc[plant, res] = res_be

res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/tcoal)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_steel.csv', header=True, index=False)

# Heat Process

In [11]:

CONV_MWh_to_MJ = 3600 #MJ/MWh
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
# GF: Glass Furnace
GFCAPEX = 1340000 #$/MWth
GFLT = 12 # years
WACC = 0.08
#costs = (model.pHeatDem/(CONV_MWh_to_MJ*365))*model.pGFCAPEX*gf_crf#*(1+model.pGFFOM)

In [16]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [18]:
heat_process_df = pd.read_excel('./h2_demand_industry_heat.xlsx', sheet_name='max')
plant_ids = list(heat_process_df['FACILITY_ID'])

In [24]:
res_be_df= pd.DataFrame(columns=['FACILITY_ID', 'Grid PEM', 'Wind unsubsidized', 'Wind subsidized', 'Solar PEC','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['FACILITY_ID'] = heat_process_df['FACILITY_ID']
res_be_df.set_index('FACILITY_ID', inplace=True)

In [25]:
from opt_deployment_heat_process import get_plant_demand, get_heat_demand
def compute_res_be_heat_process(lcoh, facility_id):
  h2_dem = get_plant_demand(facility_id)*365 #kg/year
  heat_dem = get_heat_demand(facility_id) #MJ/year
  # NG equivalent demand
  ng_dem = heat_dem*CONV_MJ_TO_MMBTU #MMBtu/year
  # H2 Furnace CAPEX
  gf_crf = WACC / (1 - (1/(1+WACC)**GFLT) ) 
  heat_power_dem = heat_dem/(CONV_MWh_to_MJ*365*24) #MWth
  capex_h2_bf = heat_power_dem*GFCAPEX*gf_crf #$/year
  # BE
  be_ng = (capex_h2_bf + lcoh*h2_dem)/ng_dem #$/MMBtu
  return be_ng

  

In [26]:
for facility_id in plant_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    res_be = compute_res_be_heat_process(lcoh, facility_id)
    res_be_df.loc[facility_id, res] = res_be

res_be_df

c:\Users\mgarrou\Projects\H2_ANRs\code\opt_deployment_heat_process.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return demand_kg_day
c:\Users\mgarrou\Projects\H2_ANRs\code\opt_deployment_heat_process.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return yearly_heat_demand
c:\Users\mgarrou\Projects\H2_ANRs\code\opt_deployment_heat_process.py:50: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return demand_kg_day
c:\Users\mgarrou\Projects\H2_ANRs\code\opt_deployment_heat_process.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return yearly_heat_demand
c:\Users\mgarrou\Projects\H2_A

,Grid PEM,Wind unsubsidized,Wind subsidized,Solar PEC,Solar PV-E,SMR 89% CCUS
FACILITY_ID,,,,,,
1000022,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757
1000029,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757
1000056,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757
1000077,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757
1000156,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757
...,...,...,...,...,...,...
1011111,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757
1011242,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757
1011799,45.137848,55.288336,47.852081,61.720697,63.207948,20.300757


In [27]:
res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/MMBtu)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_heat_process.csv', header=True, index=False)